
# TIỀN XỬ LÝ DỮ LIỆU (PHẦN 1)

Trong lĩnh vực machine learning, việc tiền xử lý dữ liệu đóng vai trò quan trọng để xây dựng những mô hình chính xác và mạnh mẽ. Dữ liệu thô thường gặp phải tình trạng lộn xộn, thiếu sót và không nhất quán. Tiền xử lý dữ liệu bao gồm một loạt các kỹ thuật và nhiệm vụ nhằm biến đổi và tinh chỉnh dữ liệu thô thành một định dạng thích hợp, chuẩn bị cho việc phân tích và huấn luyện mô hình.

Bằng cách giải quyết các vấn đề như giá trị thiếu, các ngoại lệ và tỷ lệ biến đổi khác nhau, việc tiền xử lý đảm bảo chất lượng dữ liệu được cải thiện, dẫn đến hiệu suất mô hình cải thiện.

Trong cuốn sổ tay Jupyter này, chúng ta sẽ đào sâu vào các bước cơ bản của tiền xử lý dữ liệu. Chúng ta sẽ làm việc thông qua một bài tập thực tế bằng Python, nơi chúng ta sẽ tập trung vào 2 nhiệm vụ cơ bản:

* Xác định và xử lý giá trị bị thiếu
* Chuẩn hóa dữ liệu

Khi kết thúc bài tập này, bạn sẽ đã có được kinh nghiệm thực tế trong việc đánh giá, làm sạch và biến đổi dữ liệu thô thành một định dạng thích hợp cho việc học máy. Những kỹ năng này là nền tảng trong hành trình khoa học dữ liệu, vì chúng đặt ra sân khấu cho các kỹ thuật nâng cao hơn và việc xây dựng mô hình. Vậy thì, hãy bắt đầu và học nghệ thuật tiền xử lý dữ liệu để mở khóa tiềm năng thực sự của những nỗ lực trong lĩnh vực học máy của bạn.

### 1. Tải dữ liệu

Dữ liệu được sử dụng sẽ là dữ liệu về giá nhà, cụ thể như sau:

In [1]:
import pandas as pd

data_path = "./data/housing.csv"
df = pd.read_csv(data_path, index_col=0)
df_test = df.copy()
df.head()

FileNotFoundError: [Errno 2] No such file or directory: './data/housing.csv'

### 2. Xác định số phần tử bị thiếu ở mỗi cột

In [ ]:
# Liệt kê số phần tử bị thiếu ở mỗi cột
df.isna().sum()

longitude              6826
latitude               5020
housing_median_age     7711
total_rooms           13089
total_bedrooms          432
population            13206
households            15149
median_income          2145
median_house_value     5227
dtype: int64

### 3. Loại bỏ những cột có nhiều giá trị bị thiếu

**Bài tập**: Hãy viết hàm nhận vào `dataframe` và `threshold` (ngưỡng phần trăm).
Trả về `dataframe` mới sau khi đã loại bỏ hết tất cả các cột mà tỉ lệ phần
trăm giá trị bị thiếu vượt qua `threshold`.

In [ ]:
def drop_sparse_columns(df: pd.DataFrame, threshold: float) -> pd.DataFrame:


    ### BEGIN SOLUTION
    percentage = df.isna().mean()
    col = percentage[percentage > threshold].index
    return df.drop(columns = col)
    ### END SOLUTION
    pass

Ta tiến hành loại bỏ những cột có nhiều giá trị bị thiếu

In [ ]:
# Nếu cột có phần trăm giá trị bị thiếu > 60% thì sẽ bị loại bỏ
threshold = 0.6
df = drop_sparse_columns(df, threshold)

In [ ]:
# Kiểm tra với public tests
assert df.shape[1] == 6

In [ ]:
# Sau khi đã loại bỏ những cột không cần thiết do chứa quá nhiều giá trị bị thiếu
df.isna().sum()

longitude             6826
latitude              5020
housing_median_age    7711
total_bedrooms         432
median_income         2145
median_house_value    5227
dtype: int64

### 4. Lắp đầy những giá trị thiếu ở những cột còn lại

**Bài tập**: Hãy viết các hàm thực hiện điền giá trị bị thiếu vào `dataframe` ứng với
với các chiến lược sau: ***min imputation***, ***max imputation***, ***mean imputation***, ***zero imputation***.

In [ ]:
def scale_test(df: pd.DataFrame, scaler) -> pd.DataFrame:
    scaled_data = df.apply(lambda col:
                           scaler.fit_transform(col.values.reshape(-1, 1)).flatten())
    scaled_df = pd.DataFrame(scaled_data, columns=df.columns)
    return scaled_df
    pass

def fill_with_min(df: pd.DataFrame) -> pd.DataFrame:

    ### BEGIN SOLUTION
    return df.fillna(df.min())
    ### END SOLUTION
    pass

def fill_with_max(df: pd.DataFrame) -> pd.DataFrame:

    ### BEGIN SOLUTION
    return df.fillna(df.max())
    ### END SOLUTION
    pass

def fill_with_mean(df: pd.DataFrame) -> pd.DataFrame:

    ### BEGIN SOLUTION
    return df.fillna(df.mean())
    ### END SOLUTION
    pass

def fill_with_zero(df: pd.DataFrame) -> pd.DataFrame:

    ### BEGIN SOLUTION
    return df.fillna(0)
    ### END SOLUTION
    pass


Ta gọi hàm và tạo những `dataframe` mới ứng với từng kiểu điền rỗng

In [ ]:
min_filled_df = fill_with_min(df)
max_filled_df = fill_with_max(df)
mean_filled_df = fill_with_mean(df)
zero_filled_df = fill_with_zero(df)

In [ ]:
# Kiểm tra với public tests
assert not min_filled_df.isna().any().any()
assert not max_filled_df.isna().any().any()
assert not mean_filled_df.isna().any().any()
assert not zero_filled_df.isna().any().any()

## 4. Chuẩn hóa dữ liệu
Các đặc trưng thường đi kèm với các tỷ lệ biến đổi khác nhau, điều này có thể dẫn đến mô hình thiên vị. Chúng ta sẽ khám phá các kỹ thuật chuẩn hóa phổ biến

- Min-Max Scaling: Nó biến đổi các giá trị trong tập dữ liệu về các giá trị trong khoảng từ 0 đến 1.
$$ x_{scaled} = {x-x_{min} \over x_{max} - x_{min}} $$


>>>| x | $x_{scaled}$ |
|:--------:|:--------:|
| 10       | 0.0      |
| -20       | 0.5      |
| 35       | 0.25      |
| 48       | 1.0      |
| 53       | 0.75      |

- Standard Scaling (Z-score normalization): Nó tính toán giá trị trung bình và độ lệch chuẩn của tập dữ liệu và chuẩn hóa nó bằng cách trừ giá trị trung bình và chia cho độ lệch chuẩn.

$$ x_{scaled} = {x- mean_x \over std_x} $$

>>>| x | $x_{scaled}$ |
|:--------:|:--------:|
| 10       | -0.56     |
| -20       | -1.67      |
| 35       | 0.36      |
| 48       | 0.84     |
| 53       | 1.03      |

 >>>$mean_x=$25.2, $std_x \approx$27.0658

 >>>$mean_{x_{scaled}} \approx$0, $std_{x_{scaled}} \approx$1

- Robust Scaling: RobustScaler là một kỹ thuật sử dụng trung vị và quartiles để giải quyết các bias từ các giá trị ngoại lệ.

$$ x_{scaled} = {x-x_{median} \over x_{75} - x_{25}} $$

>>>| x | $x_{scaled}$ |
|:--------:|:--------:|
| 10       | -0.66     |
| -20       | -1.45      |
| 35       | 0.0      |
| 48       | 0.34     |
| 53       | 0.47      |

![anh](https://i.imgur.com/MARX2bg.png)

 Những kỹ thuật này sẽ giúp đưa các đặc trưng về một tỷ lệ chung, ngăn chặn bất kỳ đặc trưng nào chiếm ưu thế trong quá trình học.

**Bài tập**: Hãy viết hàm nhận vào `dataframe` và một `object` thuộc một trong ba
scaler đã được import bên dưới và trả vể dataframe đã được chuẩn hóa sử dụng scaler đó.

In [ ]:
# Sử dụng các class scaler có từ thư viện sklearn
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
import numpy as np

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Tạo DataFrame với một cột dữ liệu
data = {'Column1': [10, -20, 35, 48, 53]}
df = pd.DataFrame(data)

# Khởi tạo StandardScaler
scaler = RobustScaler()

# Áp dụng StandardScaler vào cột dữ liệu
scaled_data = df.apply(lambda col: scaler.fit_transform(col.values.reshape(-1, 1)).flatten())

# In kết quả
print(scaled_data)

    Column1
0 -0.657895
1 -1.447368
2  0.000000
3  0.342105
4  0.473684


In [ ]:
def scale(df: pd.DataFrame, scaler) -> pd.DataFrame:

    ### BEGIN SOLUTION
    scaler_arr = scaler.fit_transform(df)

    scaled_df = pd.DataFrame(scaler_arr, index = df.index, columns = df.columns)

    return scaled_df

    ### END SOLUTION
    pass


Tiến hành tạo các `dataframe` ứng với từng kiểu chuẩn hóa

In [ ]:
minmax_scaled_df = scale(mean_filled_df, MinMaxScaler())
standard_scaled_df = scale(mean_filled_df, StandardScaler())
robust_scaled_df = scale(mean_filled_df, RobustScaler())